In [ ]:
from scipy.stats import special_ortho_group
import numpy as np
from kooplearn import EDMD
from kooplearn.data.datasets.stochastic import LinearModel
from rich.progress import track

In [ ]:
rng = np.random.default_rng(0)

true_rank = 5
dim = 20

def make_linear_system(true_rank = 5, dim = 20, noise = 1e-3, rng_seed = 0):
    eigs = 9*np.logspace(-8, -1, true_rank)
    print("Eigenvalues:")
    for ev in eigs:
        print(f"{ev:.1e} \t", end='')
    eigs = np.concatenate([eigs, np.zeros(dim-true_rank)])
    Q = special_ortho_group(dim, 0).rvs(1)
    A = np.linalg.multi_dot([Q, np.diag(eigs), Q.T])

    #Consistency-check
    assert np.allclose(np.sort(np.linalg.eigvalsh(A)), np.sort(eigs))
    return LinearModel(A, noise, rng_seed)

### Principal component regression:

In [ ]:
lin_sys = make_linear_system(true_rank, dim)

Eigenvalues:
9.0e-08 	5.1e-06 	2.8e-04 	1.6e-02 	9.0e-01 	

In [ ]:
num_init_cond = 100
evol_time = 100
X0 = rng.normal(size=(num_init_cond, dim))
lin_sys.generate(X0.T, evol_time).shape

def eval_EDMD_on_lin_sys( lin_sys, evol_time = 100, num_init_cond = 100, EDMD_kwargs = {}):
    X0 = rng.normal(size=(num_init_cond, dim))

    train_trajectories = lin_sys.generate(X0.T, evol_time)
    train_trajectories = np.transpose(train_trajectories, (2, 0, 1)) # (num_init_cond, evol_time, dim)
    
    estimators = []
    fro_sq_errors = []

    for traj in track(train_trajectories, description="Fitting EDMD"):
        edmd = EDMD(**EDMD_kwargs)
        edmd.fit(traj[:-1], traj[1:])

        A_estim = np.linalg.multi_dot([edmd.U, edmd.U.T, edmd.cov_XY])
        A_true = lin_sys.A
        fro_sq_errors.append((dim**-1.)*np.linalg.norm(A_estim - A_true, ord='fro')**2)
        estimators.append(A_estim)
        return estimators, fro_sq_errors
    estimators = np.array(estimators)
    fro_sq_errors = np.array(fro_sq_errors)
    ensemble_estimator = np.mean(estimators, axis=0)
    fro_sq_ensamble_error = (dim**-1.)*np.linalg.norm(lin_sys.A - ensemble_estimator, ord='fro')**2
    fro_sq_error = np.mean(fro_sq_errors)
    return fro_sq_error, fro_sq_ensamble_error

In [5]:
#PCR, Well Specified Rank, Least Squares
EDMD_kwargs = {
    'reduced_rank': False,
    'rank': true_rank,
    'tikhonov_reg': 0.,
    'svd_solver': 'full',
    'rng_seed': 0
}
fro_sq_error, fro_sq_ensamble_error = eval_EDMD_on_lin_sys(lin_sys, EDMD_kwargs=EDMD_kwargs, num_init_cond=2)

Output()